<a href="https://colab.research.google.com/github/HNR1/Bachelors-project/blob/main/diffusers_test3-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/HNR1/pytorch-fid.git
!pip install transformers diffusers tomesd accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/HNR1/pytorch-fid.git to /tmp/pip-req-build-a8jm5iac
  Running command git clone --filter=blob:none --quiet https://github.com/HNR1/pytorch-fid.git /tmp/pip-req-build-a8jm5iac
  Resolved https://github.com/HNR1/pytorch-fid.git to commit 2796969f9c1c2f979af566682b72de7f44f6e8d6
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-fid: filename=pytorch_fid-0.3.0-py3-none-any.whl size=15636 sha256=675ef198fec278726d2fb620292fc0c0e1604d3c4cc3f4190fae505338984cd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lo2th40/wheels/b2/2b/ec/2e88afc5c790068df77cc5925b988a4d23258f26104c7eda95
Successfully built pytorch-fid
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB

In [2]:
from diffusers import DiffusionPipeline
import torch, tomesd
from pytorch_fid.fid_score import calc_fid_given_lists
import os.path
from time import time

In [20]:
if os.path.exists("/content/Bachelors-project"):
  %cd Bachelors-project
  !git pull
  %cd ..
else:
  !git clone https://github.com/HNR1/Bachelors-project.git

/content/Bachelors-project
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 1.35 KiB | 1.35 MiB/s, done.
From https://github.com/HNR1/Bachelors-project
   61e2266..930ee4c  main       -> origin/main
Updating 61e2266..930ee4c
Fast-forward
 prompts.txt       | 2 +-
 seeds_768x768.txt | 3 +++
 2 files changed, 4 insertions(+), 1 deletion(-)
 create mode 100644 seeds_768x768.txt
/content


In [23]:
# parse prompts from prompts.txt
filename1 = "Bachelors-project/prompts.txt"
filename2 = "Bachelors-project/seeds_768x768.txt"
prompts = []
seeds = []
with open(filename1, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        prompts.append(line.rstrip())
with open(filename2, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        seeds.append(int(line.rstrip()))

In [5]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipeline.enable_attention_slicing()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [24]:
times_clean, times_20 = [], []
images_clean, images_20 = [], []
for (prompt, seed) in zip(prompts, seeds):
  t1 = time()
  image = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
  t2 = time()
  times_clean.append(t2-t1)
  images_clean.append(image)
  tomesd.apply_patch(pipeline, 0.2)
  t3 = time()
  image = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
  t4 = time()
  times_20.append(t4-t3)
  images_20.append(image)
  tomesd.remove_patch(pipeline)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [47]:
calc_fid_given_lists(images_clean, images_20)

123.60087892890465

In [37]:
times_clean, times_20

([92.97258281707764, 94.55265188217163, 94.35969758033752],
 [81.07378101348877, 81.63338017463684, 81.31175065040588])